In [1]:
import tkinter as tk
from tkinter import filedialog, simpledialog, messagebox
from PIL import Image, ImageTk
import math
import pandas as pd
import os

class TumorVolumeCalculator:
    def __init__(self, root):
        self.root = root
        self.root.title("Tumor Volume Calculator")

        self.front_image = None
        self.side_image = None
        self.front_image_name = None
        self.scale_front = None
        self.scale_side = None
        self.length = None
        self.width = None
        self.height = None
        self.excel_path = None

        self.setup_ui()

    def setup_ui(self):
        # Buttons
        self.load_front_button = tk.Button(self.root, text="Load Front View", command=self.load_front_image)
        self.load_front_button.pack()
        
        self.load_side_button = tk.Button(self.root, text="Load Side View", command=self.load_side_image)
        self.load_side_button.pack()

        self.calculate_button = tk.Button(self.root, text="Calculate Volume (with Side View)", command=self.calculate_volume)
        self.calculate_button.pack()
        
        self.calculate_front_only_button = tk.Button(self.root, text="Calculate Volume (Front View Only)", command=self.calculate_front_only_volume)
        self.calculate_front_only_button.pack()

        self.restart_button = tk.Button(self.root, text="Restart Calculation", command=self.restart_calculation)
        self.restart_button.pack()

        # Canvas for images
        self.front_canvas = tk.Canvas(self.root, width=500, height=500, bg='white')
        self.front_canvas.pack(side=tk.LEFT)
        self.side_canvas = tk.Canvas(self.root, width=500, height=500, bg='white')
        self.side_canvas.pack(side=tk.RIGHT)

        # Info display
        self.info_label = tk.Label(self.root, text="Follow the steps to calculate tumor volume.")
        self.info_label.pack()

    def load_front_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.front_image_name = os.path.basename(file_path)
            self.front_image = Image.open(file_path)
            self.front_image = self.resize_image(self.front_image, 800)  # Resize image
            self.front_photo = ImageTk.PhotoImage(self.front_image)
            self.front_canvas.create_image(0, 0, anchor=tk.NW, image=self.front_photo)
            self.select_scale(self.front_canvas, "front")
    
    def load_side_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.side_image = Image.open(file_path)
            self.side_image = self.resize_image(self.side_image, 800)  # Resize image
            self.side_photo = ImageTk.PhotoImage(self.side_image)
            self.side_canvas.create_image(0, 0, anchor=tk.NW, image=self.side_photo)
            self.select_scale(self.side_canvas, "side")
    
    def resize_image(self, image, max_size):
        # Proportional resizing based on width and height
        scale = min(max_size / image.width, max_size / image.height)
        resized_image = image.resize((int(image.width * scale), int(image.height * scale)))
        return resized_image

    def select_scale(self, canvas, view):
        points = []

        def on_click(event):
            points.append((event.x, event.y))
            canvas.create_oval(event.x-3, event.y-3, event.x+3, event.y+3, fill='red')
            if len(points) == 2:
                real_distance = simpledialog.askfloat("Input", "Enter real-world distance in cm:")
                pixel_distance = math.hypot(points[1][0] - points[0][0], points[1][1] - points[0][1])
                scale = real_distance / pixel_distance
                if view == "front":
                    self.scale_front = scale
                else:
                    self.scale_side = scale
                self.info_label.config(text=f"Scale for {view} view set to {scale:.4f} cm/pixel.")
                canvas.unbind('<Button-1>')
                self.select_measurement(canvas, view)

        canvas.bind('<Button-1>', on_click)

    def select_measurement(self, canvas, view):
        measurements = []
        labels = ["length", "width"] if view == "front" else ["height"]

        def on_click(event):
            measurements.append((event.x, event.y))
            canvas.create_oval(event.x-3, event.y-3, event.x+3, event.y+3, fill='blue')
            if len(measurements) % 2 == 0:
                p1, p2 = measurements[-2], measurements[-1]
                pixel_distance = math.hypot(p2[0] - p1[0], p2[1] - p1[1])
                scale = self.scale_front if view == "front" else self.scale_side
                real_distance = pixel_distance * scale
                dimension = labels[(len(measurements)//2)-1]
                if dimension == "length":
                    self.length = real_distance
                elif dimension == "width":
                    self.width = real_distance
                elif dimension == "height":
                    self.height = real_distance
                canvas.create_line(p1[0], p1[1], p2[0], p2[1], fill='green', width=2)
                self.info_label.config(text=f"{dimension.capitalize()} measured: {real_distance:.2f} cm.")
                if len(measurements) == len(labels) * 2:
                    canvas.unbind('<Button-1>')

        canvas.bind('<Button-1>', on_click)

    def calculate_volume(self):
        if self.length and self.width and self.height:
            volume = (4/3) * math.pi * (self.length / 2) * (self.width / 2) * (self.height / 2)
            self.info_label.config(text=f"Tumor Volume: {volume:.2f} cubic cm")
            self.save_to_excel(volume)
        else:
            messagebox.showerror("Error", "All dimensions are not measured.")

    def calculate_front_only_volume(self):
        if self.length and self.width:
            volume = (4/3) * math.pi * (self.length / 2) * (self.width / 2) * ((self.length * 2/3) / 2)  # Assume height = 2/3 length
            self.info_label.config(text=f"Tumor Volume (Front View Only): {volume:.2f} cubic cm")
            self.save_to_excel(volume)
        else:
            messagebox.showerror("Error", "Length and width are not measured.")

    def save_to_excel(self, volume):
        data = {
            'Front Image Name': [self.front_image_name],
            'Length (cm)': [self.length],
            'Width (cm)': [self.width],
            'Height (cm)': [self.height if self.height else 'N/A'],
            'Volume (cm³)': [volume]
        }
        df = pd.DataFrame(data)

        if not self.excel_path:
            self.excel_path = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx")])
            if not self.excel_path:
                return

        if os.path.exists(self.excel_path):
            existing_df = pd.read_excel(self.excel_path)
            df = pd.concat([existing_df, df], ignore_index=True)
        
        df.to_excel(self.excel_path, index=False)
        messagebox.showinfo("Saved", "Data saved to Excel successfully.")

    def restart_calculation(self):
        self.front_image = None
        self.side_image = None
        self.front_image_name = None
        self.scale_front = None
        self.scale_side = None
        self.length = None
        self.width = None
        self.height = None
        self.front_canvas.delete("all")
        self.side_canvas.delete("all")
        self.info_label.config(text="Follow the steps to calculate tumor volume.")

if __name__ == '__main__':
    root = tk.Tk()
    app = TumorVolumeCalculator(root)
    root.mainloop()


2025-02-25 15:03:25.238 python[32922:5813949] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-25 15:03:25.238 python[32922:5813949] +[IMKInputSession subclass]: chose IMKInputSession_Modern
2025-02-25 15:03:28.758 python[32922:5813949] The class 'NSOpenPanel' overrides the method identifier.  This method is implemented by class 'NSWindow'
2025-02-25 15:04:27.674 python[32922:5813949] The class 'NSSavePanel' overrides the method identifier.  This method is implemented by class 'NSWindow'
